### 생성 데이터 컬럼
> 위치 및 충전기 정보
- 충전소위치  충전기 <br/>	
> 충전기 정보 상세
- 충전기타입  충전기상태  충전기용량  방전지원여부  충전량  요청충전량  방전량  요청방전량<br>  
> 시간 정보
- 연결시작시간  연결종료시간  충전시작시간  충전종료시간  출방예상시간<br/>
> 차량 Soc 정보
- 베터리용량  시작값  종료값  연비

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta,timezone
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats import gaussian_kde


In [407]:
np.random.seed(42)
random.seed(42)

evse_dict = defaultdict(list)
#충전소위치
st_number = 20  # 충전소 개수
locations = [f'st-{str(n).zfill(2)}' for n in range(st_number)]
#충전기 Electric Vehicle Supply Equipment
evse_info = {}
evse_types = ['FC', 'SC']
fc_evse_volume_list = [50, 100, 200]
sc_evse_volume_list = [10, 30, 50]
evse_names = []

for loc in locations:
    evse_n = random.randint(2,6)
    for i in range(evse_n):
        evse_name = f'{loc}_evse-{str(i).zfill(2)}'
        evse_type = random.choices(['FC', 'SC'], weights=[0.3, 0.7])[0]
        if evse_type =='FC':
            volume = random.choice(fc_evse_volume_list)
        else:
            volume = random.choice(sc_evse_volume_list)
            
        evse_names.append(evse_name)
        evse_dict[loc].append(evse_name)
        evse_info[evse_name] = {
            'type': evse_type,
            'volume': volume
        }
evse_dict = defaultdict(list)

for evse in evse_names:
    loc = evse.split('_')[0]
    evse_dict[loc].append(evse)


evse_status_list = ['Available']

yes_no = ['y', 'n']
battery_capacities = [50, 60, 75, 90, 100]



# 분포 기반 함수
def generate_battery_percent(start=True, delivered_energy=None, capacity=100):
    if start:
        return round(np.clip(np.random.beta(2, 5) * 100, 5, 60), 1)
    else:
        if delivered_energy is None:
            return round(np.clip(np.random.beta(5, 2) * 100, 30, 100), 1)
        # 종료 배터리 퍼센트 = 시작 + 충전량 / 용량 * 100 (대략)
        increase = min((delivered_energy / capacity) * 100 + np.random.normal(0, 3), 100)
        return round(np.clip(increase, 5, 100), 1)

    
def generate_energy_usage(evse_volume,request_energy):
    if request_energy == 0: #충전요청이 없을 경우
        if np.random.rand() < 0.2: # 2퍼 확률로 충전전류 생성
            return round(random.uniform(0,10), 2) # 충전 요청 없었지만 충전된 경우
        return  0
    offset = request_energy*0.037
    return round(random.uniform(request_energy,request_energy + offset),2)

def generate_requested_energy():
    if np.random.rand() < 0.008:  # 예:0.5% 확률로 0 생성
        return 0.0
    val = np.random.lognormal(mean=3.1, sigma=0.25)  # 중앙값 약 27~35 부근
    return round(val, 2)
    
def generate_discharge(supports_discharge):
    if supports_discharge == 'n':
        return 0, 0
    discharged = int(np.random.gamma(2, 5))
    requested = discharged + random.randint(-3, 5)
    return max(discharged, 0), max(requested, 0)

def generate_charge_duration(evse_type):
    if evse_type == 'FC':
        return timedelta(minutes=random.randint(15, 45))
    else:  # SC
        return timedelta(minutes=random.randint(60, 180))


evse_last_end_time = {}

def generate_times(evse_type, evse_name):
    # 마지막 종료시간 이후 시작
    base_time = evse_last_end_time.get(evse_name, datetime(2019, 11, 1, tzinfo=timezone.utc))
    start = base_time + timedelta(minutes=random.randint(30, 180))  # 여유 시간 포함

    charge_delay = timedelta(minutes=random.randint(1,30))
    charge_duration = generate_charge_duration(evse_type)
    conn_duration = charge_delay + charge_duration + timedelta(minutes=random.randint(1, 15))

    chg_start = start + charge_delay
    chg_end = chg_start + charge_duration
    conn_end = start + conn_duration
    last_end = evse_last_end_time.get(evse_name, start - timedelta(hours=12))  # 직전 기록 없을 때 임의

    # 사용된 종료시간 업데이트
    evse_last_end_time[evse_name] = conn_end

    return start, conn_end, chg_start, chg_end, last_end

def generate_departure(chg_end):
    if random.random() < 0.15:
        return None  # 출발예상시간 미입력 케이스

    if random.random() < 0.8:
        # ✅ 80%: 충전 종료 이후 출발 (자연스러운 분포)
        choice = random.random()
        if choice < 0.6:
            delay = np.random.normal(20, 10)  # 일반적인 경우
        elif choice < 0.85:
            delay = np.random.exponential(30)  # 긴 꼬리
        else:
            delay = np.random.normal(45, 15)  # 길게 대기
        delay = round(np.clip(delay, 1, 120))  # 1~120분 제한
        return chg_end + timedelta(minutes=delay)
    else:
        # ✅ 20%: 충전 종료 전 출발 예정
        choice = random.random()
        if choice < 0.7:
            advance = np.random.normal(10, 5)
        else:
            advance = np.random.exponential(15)
        advance = round(np.clip(advance, 1, 60))  # 너무 빠르거나 늦지 않게
        return chg_end - timedelta(minutes=advance)

def generate_dataframe(evse_dict, n=200):
    data = []
    for location, evse_list in evse_dict.items():
        for evse_name in evse_list:
            
            for _ in range(n):
                evse_name = random.choice(evse_names)       # 예: 'st-01_evse-00'
                location = evse_name.split('_')[0]  
                evse_type = evse_info[evse_name]['type']
                evse_status = 'Available'
                evse_volume = evse_info[evse_name]['volume']
                discharge_supported = 'y' if evse_type == 'SC' and random.random() < 0.5 else 'n'
                scheduled_charge = random.choices(yes_no, weights=[0.3, 0.7])[0]

                conn_start, conn_end, chg_start, chg_end, last_end = generate_times(evse_type,evse_name)
                departure_time = generate_departure(chg_end)

                
                requested_energy = generate_requested_energy()
                delivered_energy = generate_energy_usage(evse_volume,requested_energy)
                battery_capacity = random.choice(battery_capacities)
                battery_start = generate_battery_percent(start=True, capacity=battery_capacity)
                battery_end = generate_battery_percent(start=False, delivered_energy=delivered_energy, capacity=battery_capacity)

                discharge, requested_discharge = generate_discharge(discharge_supported)
                efficiency = round(np.clip(np.random.normal(5.5, 0.8), 3.5, 7.5), 2)

                data.append({
                    '충전소위치': location,
                    '충전기이름': evse_name,
                    '충전기타입': evse_type,
                    '충전기상태': evse_status,
                    '충전기용량': evse_volume,
                    '방전지원여부': discharge_supported,
                    '예약충전': scheduled_charge,
                    '충전량(kWh)': delivered_energy,
                    '요청충전량(kWh)': requested_energy,
                    '방전량(kWh)': discharge,
                    '요청방전량(kWh)': requested_discharge,
                    '마지막충전종료시간': last_end.isoformat(),
                    '연결시작시간': conn_start.isoformat(),
                    '충전시작시간': chg_start.isoformat(),
                    '충전종료시간': chg_end.isoformat(),
                    '연결종료시간': conn_end.isoformat(),
                    '출발예상시간': departure_time.isoformat() if departure_time else None,
                    '베터리용량(kWh)': battery_capacity,
                    '시작베터리%': battery_start,
                    '종료베터리%': battery_end,
                    '연비(Wh/km)': efficiency
                })
    return pd.DataFrame(data)

df_clean = generate_dataframe(evse_dict= evse_dict, n=200)
df_clean

,충전소위치,충전기이름,충전기타입,충전기상태,충전기용량,방전지원여부,예약충전,충전량(kWh),요청충전량(kWh),방전량(kWh),...,마지막충전종료시간,연결시작시간,충전시작시간,충전종료시간,연결종료시간,출발예상시간,베터리용량(kWh),시작베터리%,종료베터리%,연비(Wh/km)
0,st-14,st-14_evse-04,SC,Available,10,n,n,21.85,21.44,0,...,2019-10-31T13:21:00+00:00,2019-11-01T01:21:00+00:00,2019-11-01T01:44:00+00:00,2019-11-01T03:23:00+00:00,2019-11-01T03:30:00+00:00,2019-11-01T03:48:00+00:00,50,10.1,44.5,6.31
1,st-10,st-10_evse-00,SC,Available,30,y,n,19.36,19.20,14,...,2019-10-31T15:00:00+00:00,2019-11-01T03:00:00+00:00,2019-11-01T03:18:00+00:00,2019-11-01T04:47:00+00:00,2019-11-01T04:57:00+00:00,2019-11-01T05:24:00+00:00,50,17.1,33.0,4.28
2,st-04,st-04_evse-01,FC,Available,50,n,n,15.92,15.59,0,...,2019-10-31T13:41:00+00:00,2019-11-01T01:41:00+00:00,2019-11-01T02:03:00+00:00,2019-11-01T02:33:00+00:00,2019-11-01T02:37:00+00:00,2019-11-01T03:04:00+00:00,60,13.9,26.9,5.09
3,st-15,st-15_evse-05,SC,Available,10,n,y,19.58,19.10,0,...,2019-10-31T12:54:00+00:00,2019-11-01T00:54:00+00:00,2019-11-01T01:16:00+00:00,2019-11-01T03:11:00+00:00,2019-11-01T03:17:00+00:00,2019-11-01T03:17:00+00:00,50,48.0,44.7,5.07
4,st-03,st-03_evse-01,SC,Available,50,y,n,27.74,27.27,14,...,2019-10-31T12:57:00+00:00,2019-11-01T00:57:00+00:00,2019-11-01T01:05:00+00:00,2019-11-01T02:29:00+00:00,2019-11-01T02:33:00+00:00,2019-11-01T02:49:00+00:00,50,5.0,59.1,5.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15195,st-15,st-15_evse-03,FC,Available,200,n,n,38.12,37.12,0,...,2019-11-23T21:23:00+00:00,2019-11-24T00:13:00+00:00,2019-11-24T00:24:00+00:00,2019-11-24T01:07:00+00:00,2019-11-24T01:17:00+00:00,2019-11-24T00:52:00+00:00,100,25.4,39.3,6.21
15196,st-11,st-11_evse-01,FC,Available,50,n,y,12.90,12.69,0,...,2019-11-24T09:54:00+00:00,2019-11-24T11:06:00+00:00,2019-11-24T11:36:00+00:00,2019-11-24T11:58:00+00:00,2019-11-24T12:11:00+00:00,2019-11-24T12:23:00+00:00,100,34.6,13.4,6.08
15197,st-16,st-16_evse-00,SC,Available,50,y,y,17.61,17.43,24,...,2019-12-12T10:00:00+00:00,2019-12-12T12:40:00+00:00,2019-12-12T12:45:00+00:00,2019-12-12T14:13:00+00:00,2019-12-12T14:23:00+00:00,2019-12-12T14:48:00+00:00,100,55.0,14.0,6.09
15198,st-07,st-07_evse-02,FC,Available,200,n,n,23.65,23.34,0,...,2019-11-23T21:31:00+00:00,2019-11-24T00:02:00+00:00,2019-11-24T00:30:00+00:00,2019-11-24T00:57:00+00:00,2019-11-24T00:58:00+00:00,2019-11-24T01:23:00+00:00,100,13.1,27.1,6.88


In [417]:
df_clean[df_clean['충전소위치']=='st-04']

,충전소위치,충전기이름,충전기타입,충전기상태,충전기용량,방전지원여부,예약충전,충전량(kWh),요청충전량(kWh),방전량(kWh),...,마지막충전종료시간,연결시작시간,충전시작시간,충전종료시간,연결종료시간,출발예상시간,베터리용량(kWh),시작베터리%,종료베터리%,연비(Wh/km)
2,st-04,st-04_evse-01,FC,Available,50,n,n,15.92,15.59,0,...,2019-10-31T13:41:00+00:00,2019-11-01T01:41:00+00:00,2019-11-01T02:03:00+00:00,2019-11-01T02:33:00+00:00,2019-11-01T02:37:00+00:00,2019-11-01T03:04:00+00:00,60,13.9,26.9,5.09
9,st-04,st-04_evse-00,FC,Available,200,n,n,22.37,22.30,0,...,2019-10-31T13:30:00+00:00,2019-11-01T01:30:00+00:00,2019-11-01T01:43:00+00:00,2019-11-01T02:01:00+00:00,2019-11-01T02:16:00+00:00,2019-11-01T02:30:00+00:00,100,54.7,19.9,5.10
11,st-04,st-04_evse-01,FC,Available,50,n,y,31.51,30.52,0,...,2019-11-01T02:37:00+00:00,2019-11-01T05:31:00+00:00,2019-11-01T05:35:00+00:00,2019-11-01T05:52:00+00:00,2019-11-01T06:01:00+00:00,2019-11-01T06:32:00+00:00,60,22.0,51.8,5.18
53,st-04,st-04_evse-02,SC,Available,10,n,y,20.09,19.44,0,...,2019-10-31T14:16:00+00:00,2019-11-01T02:16:00+00:00,2019-11-01T02:36:00+00:00,2019-11-01T04:52:00+00:00,2019-11-01T05:05:00+00:00,2019-11-01T05:07:00+00:00,75,27.6,26.2,7.07
62,st-04,st-04_evse-03,SC,Available,30,y,n,14.36,14.14,10,...,2019-10-31T14:13:00+00:00,2019-11-01T02:13:00+00:00,2019-11-01T02:41:00+00:00,2019-11-01T04:46:00+00:00,2019-11-01T05:00:00+00:00,2019-11-01T04:58:00+00:00,60,22.1,20.5,4.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15145,st-04,st-04_evse-03,SC,Available,30,n,y,20.70,20.69,0,...,2019-12-04T04:53:00+00:00,2019-12-04T06:17:00+00:00,2019-12-04T06:28:00+00:00,2019-12-04T08:40:00+00:00,2019-12-04T08:52:00+00:00,2019-12-04T08:43:00+00:00,90,18.0,28.6,5.52
15149,st-04,st-04_evse-04,FC,Available,50,n,n,16.26,15.74,0,...,2019-11-21T02:36:00+00:00,2019-11-21T04:30:00+00:00,2019-11-21T04:31:00+00:00,2019-11-21T04:46:00+00:00,2019-11-21T04:54:00+00:00,2019-11-21T05:25:00+00:00,100,45.9,15.0,5.34
15172,st-04,st-04_evse-01,FC,Available,50,n,y,28.71,27.74,0,...,2019-11-25T20:53:00+00:00,2019-11-25T22:00:00+00:00,2019-11-25T22:27:00+00:00,2019-11-25T23:11:00+00:00,2019-11-25T23:22:00+00:00,2019-11-25T23:29:00+00:00,100,50.8,29.5,4.89
15182,st-04,st-04_evse-03,SC,Available,30,n,n,16.19,16.04,0,...,2019-12-04T08:52:00+00:00,2019-12-04T11:44:00+00:00,2019-12-04T12:05:00+00:00,2019-12-04T14:11:00+00:00,2019-12-04T14:15:00+00:00,2019-12-04T14:37:00+00:00,100,5.0,18.7,5.22


In [408]:
df_clean[df_clean['충전기이름']=='st-00_evse-00']

,충전소위치,충전기이름,충전기타입,충전기상태,충전기용량,방전지원여부,예약충전,충전량(kWh),요청충전량(kWh),방전량(kWh),...,마지막충전종료시간,연결시작시간,충전시작시간,충전종료시간,연결종료시간,출발예상시간,베터리용량(kWh),시작베터리%,종료베터리%,연비(Wh/km)
25,st-00,st-00_evse-00,FC,Available,100,n,n,27.51,26.84,0,...,2019-10-31T13:23:00+00:00,2019-11-01T01:23:00+00:00,2019-11-01T01:37:00+00:00,2019-11-01T02:17:00+00:00,2019-11-01T02:27:00+00:00,2019-11-01T02:34:00+00:00,100,38.2,22.1,5.79
198,st-00,st-00_evse-00,FC,Available,100,n,n,24.16,23.72,0,...,2019-11-01T02:27:00+00:00,2019-11-01T03:56:00+00:00,2019-11-01T03:59:00+00:00,2019-11-01T04:36:00+00:00,2019-11-01T04:39:00+00:00,None,75,34.2,35.2,4.05
432,st-00,st-00_evse-00,FC,Available,100,n,n,31.42,30.69,0,...,2019-11-01T04:39:00+00:00,2019-11-01T05:57:00+00:00,2019-11-01T06:11:00+00:00,2019-11-01T06:54:00+00:00,2019-11-01T07:03:00+00:00,2019-11-01T07:42:00+00:00,90,18.6,38.4,4.38
492,st-00,st-00_evse-00,FC,Available,100,n,n,18.01,17.44,0,...,2019-11-01T07:03:00+00:00,2019-11-01T08:37:00+00:00,2019-11-01T08:38:00+00:00,2019-11-01T09:07:00+00:00,2019-11-01T09:15:00+00:00,2019-11-01T09:18:00+00:00,60,32.8,34.0,6.56
650,st-00,st-00_evse-00,FC,Available,100,n,n,14.47,14.34,0,...,2019-11-01T09:15:00+00:00,2019-11-01T11:03:00+00:00,2019-11-01T11:14:00+00:00,2019-11-01T11:47:00+00:00,2019-11-01T11:49:00+00:00,None,60,15.3,23.3,5.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15031,st-00,st-00_evse-00,FC,Available,100,n,y,20.03,19.95,0,...,2019-11-21T09:00:00+00:00,2019-11-21T11:36:00+00:00,2019-11-21T11:42:00+00:00,2019-11-21T12:02:00+00:00,2019-11-21T12:12:00+00:00,None,60,15.1,30.8,4.81
15032,st-00,st-00_evse-00,FC,Available,100,n,n,18.97,18.35,0,...,2019-11-21T12:12:00+00:00,2019-11-21T13:55:00+00:00,2019-11-21T14:19:00+00:00,2019-11-21T15:03:00+00:00,2019-11-21T15:07:00+00:00,2019-11-21T15:18:00+00:00,60,10.7,33.4,5.74
15037,st-00,st-00_evse-00,FC,Available,100,n,n,23.32,22.89,0,...,2019-11-21T15:07:00+00:00,2019-11-21T17:27:00+00:00,2019-11-21T17:52:00+00:00,2019-11-21T18:25:00+00:00,2019-11-21T18:28:00+00:00,2019-11-21T18:22:00+00:00,50,26.0,49.4,5.19
15052,st-00,st-00_evse-00,FC,Available,100,n,n,26.81,26.74,0,...,2019-11-21T18:28:00+00:00,2019-11-21T19:15:00+00:00,2019-11-21T19:22:00+00:00,2019-11-21T19:43:00+00:00,2019-11-21T19:55:00+00:00,2019-11-21T20:03:00+00:00,90,44.6,34.1,5.78


In [386]:
round(max(np.random.normal(1, 8), 0), 2)

6.31

⚡ 충전 상태 기반 패턴 (Status-driven) — 약 20종
- 정상 충전 완료
- 연결 후 충전 실패
- 충전 중 연결 종료
- 충전기 고장 상태
- 중간에 사용자에 의해 중단
- 충전 시작 지연
- 급속 충전 중단
- 완속 충전 연장
- 충전 불필요 (베터리 충분)

🕒 시간 흐름 패턴 (Time-driven) — 약 25종
- 야간 충전 (22시 ~ 06시)
- 오전 집중 충전 (출근 직전)
- 낮 시간 대기 후 충전
- 급속 충전으로 출발시간 전 도달
- 출발예상시간보다 늦게 충전 종료
- 연결시간 긴데 충전시간 짧음
- 충전기 대기 시간 포함된 시나리오
- 출발예상시간이 예측 충전과 연동
- 마지막충전종료시간이 기록 누락됨
- 충전 시작/종료 시간 역전됨 (오류)
- 연결시간은 있으나 실제 충전 없음
- 방전량이 음수로 기록 (센서 오류)
- 요청 충전량이 0인데 충전됨
- 예측 출발 시간보다 빠르게 종료
- 충전기는 정상인데 차량측 문제

① 방전 지원 여부 → 방전량/요청방전량 가능 여부
     └─ 'n'이면 둘 다 반드시 0
     
② 요청충전량 == 0 → 충전량은 반드시 0 (예외 시 '충전 불필요' 시나리오로 분리)

③ 충전기 상태 → 충전량에 영향
     └─ 'Faulted'이면 충전량=0
     └─ 'Available'인데 충전량 > 0 이면 충돌

④ 충전 시작/종료 시간 → 시간 역전 불가
     └─ 종료 > 시작 > 연결 > 이전 종료

⑤ 시작/종료 배터리 퍼센트 + 충전량 → 일관성 필요
     └─ 종료퍼센트 < 시작퍼센트면 충전량 = 0 or 음수

⑥ 출발예상시간 → 필수 입력 아님
     └─ 사용자 입력 없는 경우 `NaT` 또는 `None`

⑦ 충전기 타입 → 방전지원여부 제약
     └─ 대부분 FC는 방전 미지원 → y 불가



⚡ 충전 상태 기반 패턴 (Status-driven) — 약 20종
- 정상 충전 완료
- 연결 후 충전 실패
- 충전 중 연결 종료
- 충전기 고장 상태
- 중간에 사용자에 의해 중단
- 충전 시작 지연
- 급속 충전 중단
- 완속 충전 연장
- 충전 불필요 (베터리 충분)

🕒 시간 흐름 패턴 (Time-driven) — 약 25종
- 야간 충전 (22시 ~ 06시)
- 오전 집중 충전 (출근 직전)
- 낮 시간 대기 후 충전
- 급속 충전으로 출발시간 전 도달
- 출발예상시간보다 늦게 충전 종료
- 연결시간 긴데 충전시간 짧음
- 충전기 대기 시간 포함된 시나리오
- 출발예상시간이 예측 충전과 연동
- 베터리 용량 적을 때 반복 충전
- 마지막충전종료시간과 출발시간 간격 비교

🔋 배터리/충전량 기반 패턴 (Energy-driven) — 약 30종
- 시작 베터리 10%, 종료 90% (충전량 충분)
- 시작 30%, 종료 60% (부분충전)
- 시작 80%, 종료 100% (최적화)
- 종료 후 방전량 존재
- 요청방전량 > 실제방전량
- 방전 후 재충전
- 요청충전량 ≠ 실제충전량 (시간 부족 또는 제약)
- 고용량 충전기에서 30분 만에 완충
- 낮은 연비일 때 충전량 많음
- 시작베터리 높지만 방전 요구 있음

📍 위치/장비 타입 기반 패턴 (Location/type-driven) — 약 15종
- FC 타입에서 빠른 완충
- SC 타입에서 오래 연결
- 특정 충전소위치에서 고장 빈도 높음
- 용량이 작지만 빈도 높은 충전기
- FC 충전기에서 방전은 미지원
- SC 충전기에서 연비 반영 충전
- 동일 위치에서 반복 사용되는 사용자
- 충전기이름별 성능 차이 시뮬레이션

🔁 특수 상황 패턴 (Exception-driven) — 약 15종
- 방전지원 차량만 가능한 시나리오
- 마지막충전종료시간이 기록 누락됨
- 충전 시작/종료 시간 역전됨 (오류)
- 연결시간은 있으나 실제 충전 없음
- 방전량이 음수로 기록 (센서 오류)
- 요청 충전량이 0인데 충전됨
- 예측 출발 시간보다 빠르게 종료
- 충전기는 정상인데 차량측 문제